# Time Series DAGMM for Anomaly Detection

This notebook demonstrates the use of Deep Autoencoding Gaussian Mixture Model (DAGMM) adapted for time series anomaly detection.

## Key Features:
- LSTM-based encoder-decoder for temporal modeling
- Handles variable-length sequences with masking
- Time series specific reconstruction features
- GMM-based anomaly scoring
- Comprehensive evaluation metrics

## 1. Setup and Imports

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

print("Environment setup complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Import Time Series DAGMM Components

In [ ]:
from model_timeseries import TimeSeriesDaGMM
from data_loader_timeseries import (
    create_synthetic_dataset, 
    get_timeseries_loader,
    TimeSeriesDataset,
    SyntheticTimeSeriesGenerator
)
from solver_timeseries import TimeSeriesSolver
from utils_timeseries import (
    plot_sample_sequences,
    plot_reconstruction_comparison,
    plot_anomaly_scores,
    evaluate_anomaly_detection
)

print("Time Series DAGMM components imported successfully!")

## 3. Create Synthetic Time Series Dataset

In [ ]:
# Dataset parameters
N_NORMAL = 800
N_ANOMALOUS = 200
SEQ_LENGTH_RANGE = (50, 150)
N_FEATURES = 10  # Using smaller feature dimension for demo
TEST_SPLIT = 0.2

print("Creating synthetic time series dataset...")

# Create dataset
train_dataset, test_dataset = create_synthetic_dataset(
    n_normal=N_NORMAL,
    n_anomalous=N_ANOMALOUS,
    seq_length_range=SEQ_LENGTH_RANGE,
    n_features=N_FEATURES,
    test_split=TEST_SPLIT,
    normalization='standard'
)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Check label distribution
train_labels = [train_dataset[i][1].item() for i in range(len(train_dataset))]
test_labels = [test_dataset[i][1].item() for i in range(len(test_dataset))]

print(f"\nTraining set - Normal: {train_labels.count(0)}, Anomalous: {train_labels.count(1)}")
print(f"Test set - Normal: {test_labels.count(0)}, Anomalous: {test_labels.count(1)}")

## 4. Visualize Sample Sequences

In [ ]:
# Plot sample sequences
plot_sample_sequences(train_dataset, n_samples=4)
plt.suptitle('Sample Time Series from Training Set', fontsize=16)
plt.show()

# Show sequence length distribution
train_lengths = [train_dataset[i][2] for i in range(len(train_dataset))]
test_lengths = [test_dataset[i][2] for i in range(len(test_dataset))]

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(train_lengths, bins=20, alpha=0.7, label='Training')
plt.hist(test_lengths, bins=20, alpha=0.7, label='Test')
plt.xlabel('Sequence Length')
plt.ylabel('Frequency')
plt.title('Sequence Length Distribution')
plt.legend()

plt.subplot(1, 2, 2)
normal_lengths = [train_dataset[i][2] for i in range(len(train_dataset)) if train_dataset[i][1] == 0]
anomaly_lengths = [train_dataset[i][2] for i in range(len(train_dataset)) if train_dataset[i][1] == 1]
plt.hist(normal_lengths, bins=15, alpha=0.7, label='Normal')
plt.hist(anomaly_lengths, bins=15, alpha=0.7, label='Anomalous')
plt.xlabel('Sequence Length')
plt.ylabel('Frequency')
plt.title('Sequence Length by Class (Training)')
plt.legend()

plt.tight_layout()
plt.show()

## 5. Create Data Loaders

In [ ]:
# Create data loaders
BATCH_SIZE = 32

train_loader = get_timeseries_loader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

test_loader = get_timeseries_loader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False
)

print(f"Train batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

# Test a batch
for batch_idx, (sequences, labels, lengths, masks) in enumerate(train_loader):
    print(f"\nFirst batch:")
    print(f"Sequences shape: {sequences.shape}")
    print(f"Labels shape: {labels.shape}")
    print(f"Lengths shape: {lengths.shape}")
    print(f"Masks shape: {masks.shape}")
    print(f"Sample lengths: {lengths[:5].tolist()}")
    break

## 6. Initialize Time Series DAGMM Model

In [ ]:
# Model parameters
model_config = {
    'input_dim': N_FEATURES,
    'latent_dim': 16,
    'hidden_dim': 32,
    'num_layers': 2,
    'dropout': 0.2,
    'bidirectional': True,
    'n_gmm': 3
}

# Create model
model = TimeSeriesDaGMM(**model_config)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"Model created and moved to {device}")
print(f"\nModel architecture:")
print(model)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

## 7. Test Forward Pass

In [ ]:
# Test forward pass with a small batch
model.eval()

with torch.no_grad():
    # Get a small batch
    for sequences, labels, lengths, masks in train_loader:
        sequences = sequences[:4].to(device)  # Take only 4 samples
        masks = masks[:4].to(device)
        
        print(f"Input shape: {sequences.shape}")
        print(f"Mask shape: {masks.shape}")
        
        # Forward pass
        latent, decoded, z, gamma = model(sequences, masks)
        
        print(f"\nForward pass results:")
        print(f"Latent shape: {latent.shape}")
        print(f"Decoded shape: {decoded.shape}")
        print(f"Feature vector z shape: {z.shape}")
        print(f"GMM probabilities gamma shape: {gamma.shape}")
        
        # Test loss computation
        loss, energy, recon_error, cov_diag = model.loss_function(
            sequences, decoded, z, gamma, masks
        )
        
        print(f"\nLoss components:")
        print(f"Total loss: {loss.item():.4f}")
        print(f"Energy: {energy.item():.4f}")
        print(f"Reconstruction error: {recon_error.item():.4f}")
        print(f"Covariance diagonal: {cov_diag.item():.4f}")
        
        break

print("\nForward pass test completed successfully!")

## 8. Training Configuration and Setup

In [ ]:
# Training configuration
training_config = {
    'lr': 1e-3,
    'num_epochs': 20,  # Reduced for demo
    'batch_size': BATCH_SIZE,
    'input_dim': N_FEATURES,
    'latent_dim': model_config['latent_dim'],
    'hidden_dim': model_config['hidden_dim'],
    'num_layers': model_config['num_layers'],
    'dropout': model_config['dropout'],
    'bidirectional': model_config['bidirectional'],
    'n_gmm': model_config['n_gmm'],
    'lambda_energy': 0.1,
    'lambda_cov_diag': 0.005,
    'log_step': 5,
    'model_save_step': 50,
    'use_cuda': torch.cuda.is_available(),
    'model_save_path': './demo_models',
    'log_path': './demo_logs'
}

# Create solver
solver = TimeSeriesSolver(train_loader, test_loader, training_config)

print("Training setup completed!")
print(f"Configuration: {training_config}")

## 9. Model Training

In [ ]:
# Train the model
print("Starting training...")

train_losses = solver.train()

print("Training completed!")

# Plot training curves
plt.figure(figsize=(10, 6))
plt.plot(train_losses, 'b-', linewidth=2)
plt.title('Training Loss Curve', fontsize=16)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Final training loss: {train_losses[-1]:.4f}")

## 10. Model Evaluation and Anomaly Detection

In [ ]:
# Test the model
print("Starting model evaluation...")

test_metrics = solver.test(threshold_percentile=95)

print("\nTest Results:")
print("=" * 40)
for key, value in test_metrics.items():
    if not np.isnan(value):
        print(f"{key.replace('_', ' ').title()}: {value:.4f}")

# Additional detailed evaluation
solver.model.eval()
all_energies = []
all_labels = []
all_predictions = []

threshold = test_metrics['threshold']

with torch.no_grad():
    for sequences, labels, lengths, masks in test_loader:
        sequences = sequences.to(device)
        masks = masks.to(device)
        
        latent, decoded, z, gamma = solver.model(sequences, masks)
        sample_energy, _ = solver.model.compute_energy(z, size_average=False)
        
        energies = sample_energy.cpu().numpy()
        predictions = (energies > threshold).astype(int)
        
        all_energies.extend(energies)
        all_labels.extend(labels.numpy())
        all_predictions.extend(predictions)

all_energies = np.array(all_energies)
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

print(f"\nDetailed Classification Report:")
print(classification_report(all_labels, all_predictions, target_names=['Normal', 'Anomalous']))

## 11. Visualization of Results

In [ ]:
# Plot anomaly scores
plot_anomaly_scores(all_energies, all_labels, threshold)
plt.suptitle('Anomaly Detection Results', fontsize=16)
plt.show()

# Confusion matrix
cm = confusion_matrix(all_labels, all_predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Anomalous'],
            yticklabels=['Normal', 'Anomalous'])
plt.title('Confusion Matrix', fontsize=16)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Error analysis
false_positives = (all_labels == 0) & (all_predictions == 1)
false_negatives = (all_labels == 1) & (all_predictions == 0)
true_positives = (all_labels == 1) & (all_predictions == 1)
true_negatives = (all_labels == 0) & (all_predictions == 0)

print(f"\nError Analysis:")
print(f"True Negatives: {np.sum(true_negatives)}")
print(f"False Positives: {np.sum(false_positives)}")
print(f"False Negatives: {np.sum(false_negatives)}")
print(f"True Positives: {np.sum(true_positives)}")

if np.sum(false_positives) > 0:
    print(f"\nFalse Positive energies (mean): {all_energies[false_positives].mean():.4f}")
if np.sum(false_negatives) > 0:
    print(f"False Negative energies (mean): {all_energies[false_negatives].mean():.4f}")

## 12. Reconstruction Visualization

In [ ]:
# Visualize reconstructions for normal and anomalous samples
solver.model.eval()

# Find some normal and anomalous samples
normal_indices = np.where(all_labels == 0)[0][:2]
anomaly_indices = np.where(all_labels == 1)[0][:2]

plt.figure(figsize=(15, 10))

subplot_idx = 1

with torch.no_grad():
    sample_count = 0
    for sequences, labels, lengths, masks in test_loader:
        sequences = sequences.to(device)
        masks = masks.to(device)
        
        latent, decoded, z, gamma = solver.model(sequences, masks)
        
        # Move back to CPU for plotting
        sequences_cpu = sequences.cpu()
        decoded_cpu = decoded.cpu()
        masks_cpu = masks.cpu()
        
        for i in range(sequences.shape[0]):
            current_idx = sample_count + i
            
            if current_idx in normal_indices or current_idx in anomaly_indices:
                label = "Normal" if current_idx in normal_indices else "Anomalous"
                
                # Get valid length
                valid_len = masks_cpu[i].sum().item()
                orig_seq = sequences_cpu[i, :valid_len, :3]  # First 3 features
                recon_seq = decoded_cpu[i, :valid_len, :3]
                
                # Original
                plt.subplot(4, 2, subplot_idx)
                for j in range(3):
                    plt.plot(orig_seq[:, j], alpha=0.8, label=f'Feature {j+1}')
                plt.title(f'{label} - Original (Energy: {all_energies[current_idx]:.3f})')
                plt.ylabel('Value')
                if subplot_idx == 1:
                    plt.legend()
                
                # Reconstructed
                plt.subplot(4, 2, subplot_idx + 1)
                for j in range(3):
                    plt.plot(recon_seq[:, j], alpha=0.8, label=f'Feature {j+1}')
                plt.title(f'{label} - Reconstructed')
                plt.ylabel('Value')
                if subplot_idx == 1:
                    plt.legend()
                
                subplot_idx += 2
                
                if subplot_idx > 8:
                    break
        
        sample_count += sequences.shape[0]
        
        if subplot_idx > 8:
            break

plt.tight_layout()
plt.suptitle('Original vs Reconstructed Sequences', fontsize=16, y=1.02)
plt.show()

## 13. Feature Analysis

In [ ]:
# Analyze the learned features
solver.model.eval()

all_z_features = []
all_latents = []
feature_labels = []

with torch.no_grad():
    for sequences, labels, lengths, masks in test_loader:
        sequences = sequences.to(device)
        masks = masks.to(device)
        
        latent, decoded, z, gamma = solver.model(sequences, masks)
        
        all_z_features.append(z.cpu().numpy())
        all_latents.append(latent.cpu().numpy())
        feature_labels.extend(labels.numpy())

all_z_features = np.vstack(all_z_features)
all_latents = np.vstack(all_latents)
feature_labels = np.array(feature_labels)

print(f"Feature analysis:")
print(f"Latent features shape: {all_latents.shape}")
print(f"Combined features shape: {all_z_features.shape}")

# Plot feature distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Latent features (first few dimensions)
for i in range(min(3, all_latents.shape[1])):
    normal_feat = all_latents[feature_labels == 0, i]
    anomaly_feat = all_latents[feature_labels == 1, i]
    
    axes[i].hist(normal_feat, bins=30, alpha=0.7, label='Normal', density=True)
    axes[i].hist(anomaly_feat, bins=30, alpha=0.7, label='Anomalous', density=True)
    axes[i].set_title(f'Latent Feature {i+1}')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

# Temporal features (last few dimensions of z)
temporal_features = ['MSE', 'MAE', 'DTW-like', 'Trend']
for i, name in enumerate(temporal_features[:3]):
    feat_idx = all_z_features.shape[1] - 4 + i
    normal_feat = all_z_features[feature_labels == 0, feat_idx]
    anomaly_feat = all_z_features[feature_labels == 1, feat_idx]
    
    axes[i+3].hist(normal_feat, bins=30, alpha=0.7, label='Normal', density=True)
    axes[i+3].hist(anomaly_feat, bins=30, alpha=0.7, label='Anomalous', density=True)
    axes[i+3].set_title(f'Temporal Feature: {name}')
    axes[i+3].legend()
    axes[i+3].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Feature Distributions: Normal vs Anomalous', fontsize=16, y=1.02)
plt.show()

## 14. Model Interpretation and Insights

In [ ]:
# Analyze model parameters
print("Model Analysis:")
print("=" * 40)

# GMM parameters
phi = solver.model.phi.cpu().numpy()
mu = solver.model.mu.cpu().numpy()
cov = solver.model.cov.cpu().numpy()

print(f"GMM Component Weights (phi): {phi}")
print(f"Component weights sum: {phi.sum():.4f}")

print(f"\nGMM Component Means shape: {mu.shape}")
for i, component_mean in enumerate(mu):
    print(f"Component {i+1} mean magnitude: {np.linalg.norm(component_mean):.4f}")

print(f"\nGMM Covariance matrices shape: {cov.shape}")
for i, component_cov in enumerate(cov):
    eigenvals = np.linalg.eigvals(component_cov)
    print(f"Component {i+1} eigenvalue range: [{eigenvals.min():.4f}, {eigenvals.max():.4f}]")

# Feature importance analysis
print(f"\nFeature Importance Analysis:")
print("=" * 30)

# Compute feature correlations with anomaly scores
from scipy.stats import pearsonr

feature_names = [f'Latent_{i}' for i in range(all_latents.shape[1])] + \
                ['MSE', 'MAE', 'DTW_like', 'Trend']

correlations = []
for i in range(all_z_features.shape[1]):
    corr, p_val = pearsonr(all_z_features[:, i], all_energies)
    correlations.append((feature_names[i], corr, p_val))

# Sort by absolute correlation
correlations.sort(key=lambda x: abs(x[1]), reverse=True)

print("Top features correlated with anomaly scores:")
for name, corr, p_val in correlations[:8]:
    significance = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
    print(f"{name:15}: {corr:6.3f} {significance}")

## 15. Summary and Conclusions

In [ ]:
print("Time Series DAGMM Demo Summary")
print("=" * 50)

print(f"\nDataset:")
print(f"  - Training samples: {len(train_dataset)}")
print(f"  - Test samples: {len(test_dataset)}")
print(f"  - Features: {N_FEATURES}")
print(f"  - Sequence length range: {SEQ_LENGTH_RANGE}")

print(f"\nModel Architecture:")
print(f"  - LSTM Encoder: {model_config['bidirectional']} bidirectional, {model_config['num_layers']} layers")
print(f"  - Hidden dimension: {model_config['hidden_dim']}")
print(f"  - Latent dimension: {model_config['latent_dim']}")
print(f"  - GMM components: {model_config['n_gmm']}")
print(f"  - Total parameters: {total_params:,}")

print(f"\nTraining:")
print(f"  - Epochs: {training_config['num_epochs']}")
print(f"  - Learning rate: {training_config['lr']}")
print(f"  - Final loss: {train_losses[-1]:.4f}")

print(f"\nPerformance:")
for key, value in test_metrics.items():
    if not np.isnan(value):
        print(f"  - {key.replace('_', ' ').title()}: {value:.4f}")

print(f"\nKey Features:")
print(f"  ✓ Handles variable-length sequences with masking")
print(f"  ✓ LSTM-based temporal modeling")
print(f"  ✓ Time series specific reconstruction features")
print(f"  ✓ GMM-based anomaly scoring")
print(f"  ✓ End-to-end trainable architecture")

print(f"\nThis implementation successfully adapts DAGMM for time series anomaly detection,")
print(f"achieving {test_metrics['accuracy']:.1%} accuracy on the synthetic dataset.")
print(f"\nThe model can be easily extended for real-world time series applications!")